In [20]:
from numpy.linalg import norm
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt
from scipy import optimize
from scipy.sparse import csr_matrix
from scipy.optimize import minimize
from sklearn.datasets import load_iris
from sklearn.datasets import load_svmlight_file
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.utils.validation import check_is_fitted, check_array, check_X_y, check_random_state
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import ParameterGrid,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

from LMNN import LargeMarginNearestNeighbor
from LMNN_SS import SemiSupervisedLargeMarginNearestNeighbor

import time

# Metrics Learning 

**Group Members: Oskar Hint , Xiaoshen Hou,  Kasper Schønberg,  Valentine Van Der Nya**




***
## Tuning hyper-parameters

In [7]:
###Benchmark Grid Search Hyperparameters K, omega1,2, 3 #############
K=[1,2,3,4,5,6,7,8,9,10]
omega0= np.arange(0.1,1,0.1)
omega1= np.arange(0.1,1,0.1)
omega2= np.arange(0.1,1,0.1)

lmnn_param_grid = {'n_neighbors': K, 'omega': omega0}
ssc_param_grid = {'n_neighbors': K, 'omega0': omega0,'omega1': omega1,'omega2': omega2}


In [8]:
param_grid

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'omega0': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9]),
 'omega1': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9]),
 'omega2': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9])}

***
## 1.  Experinment 1 - USPS data 

In [10]:
def get_USPS_training_data():
    data = load_svmlight_file('ml_data/usps/usps')
    return data[0], data[1]

def get_USPS_testing_data():
    data = load_svmlight_file('ml_data/usps/usps.t')
    return data[0], data[1]

X_train, y_train = get_USPS_training_data()
X_test, y_test = get_USPS_testing_data()

In [11]:
######## Sample label, unlabel set  from training #########
X_train=np.asarray(X_train.todense())
y_train=np.asarray(y_train)
X_test=np.asarray(X_test.todense())
y_test=np.asarray(y_test)

sample_inds=np.random.randint(X_train.shape[0], size=200)
X_train_labelled=X_train[sample_inds, :]
y_train_labelled=y_train[sample_inds]

new_X_train = np.delete(X_train, sample_inds,0)
new_y_train = np.delete(y_train, sample_inds,0)

sample_inds=np.random.randint(new_X_train.shape[0], size=4000)
X_train_unlabelled=new_X_train[sample_inds, :]



In [22]:
##### LMNN TUNNING ########
clf = GridSearchCV(LargeMarginNearestNeighbor(), lmnn_param_grid, cv=5)
clf.fit(X_train_labelled, y_train_labelled)

print("LMNN Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

LMNN Best parameters set found on development set:

{'n_neighbors': 1, 'omega': 0.20000000000000001}

Grid scores on development set:

0.885 (+/-0.068) for {'n_neighbors': 1, 'omega': 0.10000000000000001}
0.890 (+/-0.080) for {'n_neighbors': 1, 'omega': 0.20000000000000001}
0.885 (+/-0.068) for {'n_neighbors': 1, 'omega': 0.30000000000000004}
0.885 (+/-0.068) for {'n_neighbors': 1, 'omega': 0.40000000000000002}
0.890 (+/-0.080) for {'n_neighbors': 1, 'omega': 0.5}
0.890 (+/-0.080) for {'n_neighbors': 1, 'omega': 0.59999999999999998}
0.880 (+/-0.090) for {'n_neighbors': 1, 'omega': 0.70000000000000007}
0.880 (+/-0.084) for {'n_neighbors': 1, 'omega': 0.80000000000000004}
0.880 (+/-0.084) for {'n_neighbors': 1, 'omega': 0.90000000000000002}
0.850 (+/-0.065) for {'n_neighbors': 2, 'omega': 0.10000000000000001}
0.845 (+/-0.063) for {'n_neighbors': 2, 'omega': 0.20000000000000001}
0.845 (+/-0.070) for {'n_neighbors': 2, 'omega': 0.30000000000000004}
0.840 (+/-0.067) for {'n_neighbors': 2, '

In [38]:
print("########  Labelled = ",X_train_labelled.shape, "  ###  Unlabelled = ", X_train_unlabelled.shape, "###########")
for k in K:
    print("########  K = ",k, " ###########")
    lmnn = LargeMarginNearestNeighbor(n_neighbors=k)
    lmnn.fit(X_train_labelled,y_train_labelled)
    print("USPS LMNN : ", lmnn.score(X_test,y_test))

    ssc = SemiSupervisedLargeMarginNearestNeighbor(n_neighbors=k)
    ssc.fit(X_train_labelled,X_train_unlabelled, y_train_labelled)
    print("USPS SSC : ", ssc.score(X_test,y_test))
    
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train_labelled,y_train_labelled)
    print("USPS KNN : ",  neigh.score(X_test,y_test))

########  Labelled =  (200, 256)   ###  Unlabelled =  (4000, 256) ###########
########  K =  1  ###########
USPS LMNN :  0.842551071251
USPS SSC :  0.838066766318
USPS KNN :  0.838066766318
########  K =  2  ###########
USPS LMNN :  0.828599900349
USPS SSC :  0.837568510214
USPS KNN :  0.827603388142
########  K =  3  ###########
USPS LMNN :  0.808669656203
USPS SSC :  0.809666168411
USPS KNN :  0.823119083209
########  K =  4  ###########
USPS LMNN :  0.831091180867
USPS SSC :  0.819631290483
USPS KNN :  0.825610363727
########  K =  5  ###########
USPS LMNN :  0.810164424514
USPS SSC :  0.803188839063
USPS KNN :  0.814150473343
########  K =  6  ###########
USPS LMNN :  0.800697558545
USPS SSC :  0.798206278027
USPS KNN :  0.809167912307
########  K =  7  ###########
USPS LMNN :  0.77130044843
USPS SSC :  0.788739412058
USPS KNN :  0.798206278027
########  K =  8  ###########
USPS LMNN :  0.746885899352
USPS SSC :  0.761833582461
USPS KNN :  0.794220229198
########  K =  9  #########

***
## 2.  Experinment 2 - MNIST data 

In [ ]:
mnist = fetch_mldata('MNIST original', data_home='ml_data/mnist')  #Download MMNIST

In [ ]:
X, y = mnist.data, mnist.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [ ]:
######## Sample label, unlabel set  from training #########
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
X_test=np.asarray(X_test)
y_test=np.asarray(y_test)

sample_inds=np.random.randint(X_train.shape[0], size=200)
X_train_labelled=X_train[sample_inds, :]
y_train_labelled=y_train[sample_inds]

new_X_train = np.delete(X_train, sample_inds,0)
new_y_train = np.delete(y_train, sample_inds,0)

sample_inds=np.random.randint(new_X_train.shape[0], size=4000)
X_train_unlabelled=new_X_train[sample_inds, :]

In [ ]:
K=[1,2,3,4,5,6,7,8,9,10]
for k in K:
    print("########  K = ",k, " ###########")
    lmnn = LargeMarginNearestNeighbor(n_neighbors=k)
    lmnn.fit(X_train[0:200,],y_train[0:200])
    print("MNIST LMNN : ", lmnn.score(X_test,y_test))

    ssc = SemiSupervisedLargeMarginNearestNeighbor(n_neighbors=k)
    ssc.fit(X_train[0:200,],X_train[201:600,], y_train[0:200,])
    print("MNIST SSC : ", ssc.score(X_test.todense(),y_test))
    
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train[0:200,],y_train[0:200,])
    print("MNIST KNN : ", neigh.score(X_test,y_test))

## 3. Experiment - Semi-supervised Iris

In [ ]:
dataset = load_iris()
X, y = dataset.data, dataset.target

# 25 points in labeled, 50 in unlabeled, 75 in test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_labeled = np.delete(X_train, range(50), 0)
y_labeled = np.delete(y_train, range(50))
X_unlabeled = np.delete(X_train, range(50,75), 0)

In [ ]:
K=[1,2,3,4,5,6,7,8,9,10]
for k in K:
    print("########  K = ",k, " ###########")
    time1 = time.time()
    ssc = SemiSupervisedLargeMarginNearestNeighbor(n_neighbors=k)
    ssc.fit(X_labeled, X_unlabeled, y_labeled)
    time2 = time.time()
    print("Iris SSC : ", ssc.score(y_test, ssc.predict(X_test)), " Time(s) :", (time2-time1))
   
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train,y_train)
    print("Iris KNN : ", neigh.score(X_test,y_test))